In [59]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from PIL import Image
from torchvision import transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

In [60]:
# === 2. AUGMENTATION === #
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.Normalize(),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
])

test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
    ToTensorV2(),
])

In [61]:
class WheatDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Path to the dataset folder (e.g., 'wheat-plant-diseases/train')
            transform (callable, optional): Optional transform to be applied on an image sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {}

        # Get all class names (subdirectory names)
        classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}

        # Collect all image paths and their corresponding labels
        for cls_name in classes:
            cls_folder = os.path.join(root_dir, cls_name)
            if os.path.isdir(cls_folder):
                for img_file in os.listdir(cls_folder):
                    img_path = os.path.join(cls_folder, img_file)
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[cls_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        image = np.array(image)  # Albumentations expects NumPy array
    
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]  # ToTensorV2 returns tensor here
    
        return image, label


In [62]:
train_dataset = WheatDiseaseDataset('/kaggle/input/wheat-plant-diseases/data/train', transform=train_transform)

In [63]:
valid_dataset = WheatDiseaseDataset('/kaggle/input/wheat-plant-diseases/data/valid', transform=val_transform)

In [29]:
train_dataset[0]

(tensor([[[ 1.3584,  1.3584,  1.3755,  ..., -0.0629, -0.5938, -0.2684],
          [ 1.2899,  1.3242,  1.3070,  ..., -0.2513, -0.2171, -0.1657],
          [ 1.2385,  1.2728,  1.2385,  ..., -0.2342, -0.2684, -0.4568],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[ 1.5532,  1.5882,  1.6408,  ..., -0.5301, -1.0553, -0.7052],
          [ 1.5532,  1.5882,  1.5882,  ..., -0.5126, -0.3550, -0.2850],
          [ 1.5357,  1.5532,  1.5532,  ..., -0.3725, -0.2675, -0.3725],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.4559, -1.3861, -1.3339,  ..., -1.0898, -1.3861, -0.8981],
          [-1.5604, -1.5081,

In [64]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=True)

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = models.resnet18(pretrained=True)
num_classes = len(train_dataset.class_to_idx)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

In [66]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(train_dataset.class_to_idx)

model = efficientnet_b0(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

In [67]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [68]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0

    for images, labels in tqdm(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

    return running_loss / len(loader.dataset), correct / len(loader.dataset)

In [69]:
def evaluate(model, loader, criterion):
    model.eval()
    loss = 0.0
    correct = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss += criterion(outputs, labels).item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()

    return loss / len(loader.dataset), correct / len(loader.dataset)

In [70]:
EPOCHS = 10
for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = evaluate(model, valid_loader, criterion)

        print(f"Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")


Epoch 1/10


100%|██████████| 410/410 [07:12<00:00,  1.05s/it]


Train Loss: 0.9786, Acc: 0.6890
Val   Loss: 1.5996, Acc: 0.6200

Epoch 2/10


100%|██████████| 410/410 [07:10<00:00,  1.05s/it]


Train Loss: 0.6361, Acc: 0.7890
Val   Loss: 1.0902, Acc: 0.7733

Epoch 3/10


...

Train Loss: 0.3173, Acc: 0.8875
Val   Loss: 1.0341, Acc: 0.8533

Epoch 9/10


100%|██████████| 410/410 [07:07<00:00,  1.04s/it]


Train Loss: 0.3138, Acc: 0.8867
Val   Loss: 1.5694, Acc: 0.8833

Epoch 10/10


100%|██████████| 410/410 [07:11<00:00,  1.05s/it]


Train Loss: 0.2709, Acc: 0.9029
Val   Loss: 1.4179, Acc: 0.8900


In [71]:
test_dataset = WheatDiseaseDataset('/kaggle/input/wheat-plant-diseases/data/test', transform=test_transform)

In [72]:
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [73]:
model.eval()
loss = 0.0
correct = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss += criterion(outputs, labels).item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()

print(loss / len(test_loader.dataset), correct / len(test_loader.dataset))

1.4750075495438038 0.8786666666666667


In [75]:
torch.save(model.state_dict(), f'model.pth')